## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Kharod,IN,21.7500,82.5667,82.98,70,100,12.10,overcast clouds
1,1,Manta,EC,-0.9500,-80.7333,73.18,88,75,10.36,broken clouds
2,2,Dikson,RU,73.5069,80.5464,47.50,66,100,14.65,overcast clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,33.46,75,75,6.91,broken clouds
4,4,Upata,VE,8.0086,-62.3989,74.19,95,100,4.16,overcast clouds


In [17]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
print('--------------------------------------')      
print(f'Please answer the following questions')
print('--------------------------------------\n')

min_temp=float(input('Please indicate the minimum temperature for your destination: '))
max_temp=float(input('Please indicate the maximum temperature for your destination: '))

--------------------------------------
Please answer the following questions
--------------------------------------

Please indicate the minimum temperature for your destination: 50
Please indicate the maximum temperature for your destination: 90


In [33]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_city_data_df=city_data_df.loc[(city_data_df['Max Temp']<=max_temp)&(city_data_df['Max Temp']>=min_temp)]

print(f'Nummber of results: {len(filtered_city_data_df)}')
filtered_city_data_df.head()

Nummber of results: 575


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Kharod,IN,21.7500,82.5667,82.98,70,100,12.10,overcast clouds
1,1,Manta,EC,-0.9500,-80.7333,73.18,88,75,10.36,broken clouds
4,4,Upata,VE,8.0086,-62.3989,74.19,95,100,4.16,overcast clouds
5,5,Goderich,CA,43.7501,-81.7165,66.90,77,6,10.36,clear sky
6,6,New Norfolk,AU,-42.7826,147.0587,56.71,79,98,1.99,overcast clouds


In [34]:
# 4a. Determine if there are any empty rows.
filtered_city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 575 entries, 0 to 667
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   City_ID      575 non-null    int64  
 1   City         575 non-null    object 
 2   Country      575 non-null    object 
 3   Lat          575 non-null    float64
 4   Lng          575 non-null    float64
 5   Max Temp     575 non-null    float64
 6   Humidity     575 non-null    int64  
 7   Cloudiness   575 non-null    int64  
 8   Wind Speed   575 non-null    float64
 9   Description  575 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 49.4+ KB


In [36]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
filtered_city_data_df=filtered_city_data_df.dropna()
filtered_city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 575 entries, 0 to 667
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   City_ID      575 non-null    int64  
 1   City         575 non-null    object 
 2   Country      575 non-null    object 
 3   Lat          575 non-null    float64
 4   Lng          575 non-null    float64
 5   Max Temp     575 non-null    float64
 6   Humidity     575 non-null    int64  
 7   Cloudiness   575 non-null    int64  
 8   Wind Speed   575 non-null    float64
 9   Description  575 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 49.4+ KB


In [92]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
filtered_city_data_df=filtered_city_data_df.rename(columns={'Description':'Current Description'})

hotel_df = filtered_city_data_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kharod,IN,82.98,overcast clouds,21.7500,82.5667,
1,Manta,EC,73.18,broken clouds,-0.9500,-80.7333,
4,Upata,VE,74.19,overcast clouds,8.0086,-62.3989,
5,Goderich,CA,66.90,clear sky,43.7501,-81.7165,
6,New Norfolk,AU,56.71,overcast clouds,-42.7826,147.0587,
8,Constitucion,CL,55.04,overcast clouds,-35.3333,-72.4167,
9,Saskylakh,RU,57.07,overcast clouds,71.9167,114.0833,
10,Madinat Sittah Uktubar,EG,82.00,clear sky,29.8167,31.0500,
11,Whitehorse,CA,75.97,broken clouds,60.7161,-135.0538,
12,Port Elizabeth,ZA,50.29,light rain,-33.9180,25.5701,


In [93]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print('Hotel not found...moving to next result.') 
        

Hotel not found...moving to next result.
Hotel not found...moving to next result.
Hotel not found...moving to next result.
Hotel not found...moving to next result.
Hotel not found...moving to next result.
Hotel not found...moving to next result.
Hotel not found...moving to next result.
Hotel not found...moving to next result.
Hotel not found...moving to next result.
Hotel not found...moving to next result.
Hotel not found...moving to next result.
Hotel not found...moving to next result.
Hotel not found...moving to next result.
Hotel not found...moving to next result.
Hotel not found...moving to next result.
Hotel not found...moving to next result.
Hotel not found...moving to next result.
Hotel not found...moving to next result.
Hotel not found...moving to next result.
Hotel not found...moving to next result.
Hotel not found...moving to next result.
Hotel not found...moving to next result.
Hotel not found...moving to next result.
Hotel not found...moving to next result.
Hotel not found.

In [97]:
# 7. Drop the rows where there is no Hotel Name.
no_hotel_name_index=hotel_df[(hotel_df['Hotel Name']=='')].index

clean_hotel_df=hotel_df.drop(no_hotel_name_index)

clean_hotel_df.head(20)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kharod,IN,82.98,overcast clouds,21.7500,82.5667,STARiHOTELS Sheorinarayan
1,Manta,EC,73.18,broken clouds,-0.9500,-80.7333,Balandra Hotel
4,Upata,VE,74.19,overcast clouds,8.0086,-62.3989,Posada Villa Nela
5,Goderich,CA,66.90,clear sky,43.7501,-81.7165,Harmony Inn
6,New Norfolk,AU,56.71,overcast clouds,-42.7826,147.0587,The Shingles Riverside Cottages
8,Constitucion,CL,55.04,overcast clouds,-35.3333,-72.4167,Hotel Boutique Las Azucenas
10,Madinat Sittah Uktubar,EG,82.00,clear sky,29.8167,31.0500,Happy Kennel Academy
11,Whitehorse,CA,75.97,broken clouds,60.7161,-135.0538,The Sternwheeler Hotel & Conference Centre
12,Port Elizabeth,ZA,50.29,light rain,-33.9180,25.5701,39 On Nile Guest House
13,Kubuta,SZ,52.38,clear sky,-26.8833,31.4833,Ka-Mbebe/Nyoni/Nzameya


In [99]:
# 8a. Create the output File (CSV)
save_as = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(save_as, index_label="City_ID")

In [112]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
city_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [114]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=city_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))